# YOLOv8 셋팅

In [ ]:
!pip install ultralytics

# 바다속 쓰레기 데이터셋

In [ ]:
!wget -O trash_inst_material.zip https://www.dropbox.com/s/ievh0sesad015z0/trash_inst_material.zip?dl=1

In [ ]:
!unzip trash_inst_material.zip

In [15]:
# define helper functions to show images - polygon shape
def imShow(path, lb_path):
  import cv2
  import matplotlib.pyplot as plt
  import numpy as np
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]

  file = open(lb_path, "r")
  strings = file.readlines()
  file.close()

  # Draw a rectangle with blue line borders of thickness of 2 px
  for line in strings:
    line = line.split()
    label = line[0]
    pts = []
    for index in range(1,len(line[1:])+1, 2):
      x_pt = int(width*float(line[index]))
      y_pt = int(height*float(line[index+1]))
      pts.append([x_pt,y_pt])

    pts = np.array(pts).reshape(1, -1, 2)

    image = cv2.polylines(image, pts, isClosed=True, color=(0, 255, 0), thickness=2)

  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

In [ ]:
imShow('/content/trash_inst_material/train/images/vid_000002_frame0000022.jpg','/content/trash_inst_material/train/labels/vid_000002_frame0000022.txt')

# Custom data yaml 제작

In [19]:
import yaml

data = { 'train' : '/content/trash_inst_material/train/images/',
         'val' : '/content/trash_inst_material/val/images/',
         'names' : ['rov', 'plant', 'animal_fish', 'animal_starfish',
                    'animal_shells', 'animal_crab', 'animal_eel', 'animal_etc',
                    'trash_etc', 'trash_fabric', 'trash_fishing_gear', 'trash_metal',
                    'trash_paper', 'trash_plastic', 'trash_rubber', 'trash_wood'],
         'nc' : 16 }

with open('/content/trash_inst_material/trash_inst_material.yaml', 'w') as f:
  yaml.dump(data, f)


with open('/content/trash_inst_material/trash_inst_material.yaml', 'r') as f:
  mask_wearing_yaml = yaml.safe_load(f)
  display(mask_wearing_yaml)

{'names': ['rov',
  'plant',
  'animal_fish',
  'animal_starfish',
  'animal_shells',
  'animal_crab',
  'animal_eel',
  'animal_etc',
  'trash_etc',
  'trash_fabric',
  'trash_fishing_gear',
  'trash_metal',
  'trash_paper',
  'trash_plastic',
  'trash_rubber',
  'trash_wood'],
 'nc': 16,
 'train': '/content/trash_inst_material/train/images/',
 'val': '/content/trash_inst_material/val/images/'}

In [ ]:
!yolo task=segment mode=train model=yolov8n-seg.pt imgsz=640 data=/content/trash_inst_material/trash_inst_material.yaml epochs=100 batch=16 name=yolov8n-seg exist_ok=True amp=False

Ultralytics YOLOv8.0.145 🚀 Python-3.10.6 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=/content/trash_inst_material/trash_inst_material.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8n-seg, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=False, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, 

In [ ]:
!yolo model=runs/segment/yolov8n-seg/weights/best.pt mode=predict source=trash_inst_material/val/images name=yolov8n_seg_infer_valimages exist_ok=True